<a href="https://colab.research.google.com/github/Karishma-Kuria/CMPE-255-Data-Mining/blob/main/ANNAlgorithms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Approximate nearest neighbor search using various state of art library

## Installing all necessary libraries

In [87]:
!pip install lightfm
!pip install nmslib
!pip install faiss
!pip install annoy
!pip install faiss-cpu --no-cache

## For getting the data here I am installing Stack API

In [88]:
!pip install stackapi

## **Importing all relevant libraries for the process**

In [89]:
import pickle
from lightfm import LightFM
from lightfm.datasets import fetch_stackexchange
import nmslib
import faiss
import annoy
import time
from stackapi import StackAPI

## Loading Data from the dataset

In [90]:
stackexchange_data = fetch_stackexchange('crossvalidated')
stackexchange_data

{'item_feature_labels': array(['question_id:0', 'question_id:1', 'question_id:2', ...,
        'question_id:72357', 'question_id:72358', 'question_id:72359'],
       dtype='<U17'),
 'item_features': <72360x72360 sparse matrix of type '<class 'numpy.float32'>'
 	with 72360 stored elements in Compressed Sparse Row format>,
 'test': <2836x72360 sparse matrix of type '<class 'numpy.float32'>'
 	with 8020 stored elements in COOrdinate format>,
 'train': <2836x72360 sparse matrix of type '<class 'numpy.float32'>'
 	with 51477 stored elements in COOrdinate format>}

## Dividing the data into Test and Train data.
## Model Building and training

In [91]:
train_data = stackexchange_data['train']
test_data = stackexchange_data['test']

# defining lightfm model by specifying hyper-parametre
# then using the ineteractions matrix to fit the model, item and user features 
model = LightFM(learning_rate=0.05, loss='warp', no_components=64, item_alpha=0.001)
model.fit_partial(train_data, item_features=stackexchange_data['item_features'], epochs=20 )

data_item_vectors = stackexchange_data['item_features'] * model.item_embeddings

In [92]:
stackexchange_data['item_feature_labels']

array(['question_id:0', 'question_id:1', 'question_id:2', ...,
       'question_id:72357', 'question_id:72358', 'question_id:72359'],
      dtype='<U17')

## Using pickle for dumping data with item vectors.

In [93]:
with open('stack_exchange.pickle', 'wb') as f:
    pickle.dump({"name": stackexchange_data['item_feature_labels'], "vector": item_vectors}, f)

## Loading data from the above file and Visualizing it

In [94]:
def load_data():
    with open('stack_exchange.pickle', 'rb') as f:
        data = pickle.load(f)
    return data

dataset = load_data()
dataset

{'name': array(['question_id:0', 'question_id:1', 'question_id:2', ...,
        'question_id:72357', 'question_id:72358', 'question_id:72359'],
       dtype='<U17'),
 'vector': array([[ 5.5895172e-02,  1.1990369e-01, -1.1908560e-01, ...,
          3.5561640e-02,  8.8606728e-03,  6.2211487e-02],
        [-7.5620688e-02,  3.4251142e-02, -7.7318780e-02, ...,
          9.4727725e-02, -5.9194420e-02,  5.2315751e-03],
        [-8.0306500e-02,  8.2449630e-02, -1.7578368e-01, ...,
          5.1416919e-02,  7.3899886e-05,  5.2313991e-03],
        ...,
        [-7.5524454e-03,  2.1180427e-03,  8.4224651e-03, ...,
         -4.0007071e-04,  2.3719552e-03, -3.7159508e-03],
        [ 1.2772135e-02, -3.4847087e-03,  2.0979075e-02, ...,
          2.3611756e-02,  2.4236694e-02, -3.7828584e-03],
        [-2.4579108e-02, -7.9067359e-03,  2.5011292e-02, ...,
         -5.2589890e-02,  3.6753878e-02,  3.7756007e-02]], dtype=float32)}

# **LSH**

Locality Sensitive Hashing (LSH): this algorithm works by first grouping all vectors into small buckets and then processing each vector through a hash function that maximizes hashing collisions, rather than minimizing which is usual with most of the hashing functions.
It works on the principle that if two points are closer to each other in a given space then they are supposed to have the same hash hence this puts both of them into the same bucket.
In LSH wide range of the performance depends on parameter set. Fast search results in bad quality results while slower search results in good quality.
This Search process is very simple to implement and is good for embedded system.

## Vector Visualization

In [95]:
item_vectors_value = dataset["vector"]
item_names_value = dataset["name"]

In [96]:
item_vectors_value

array([[ 5.5895172e-02,  1.1990369e-01, -1.1908560e-01, ...,
         3.5561640e-02,  8.8606728e-03,  6.2211487e-02],
       [-7.5620688e-02,  3.4251142e-02, -7.7318780e-02, ...,
         9.4727725e-02, -5.9194420e-02,  5.2315751e-03],
       [-8.0306500e-02,  8.2449630e-02, -1.7578368e-01, ...,
         5.1416919e-02,  7.3899886e-05,  5.2313991e-03],
       ...,
       [-7.5524454e-03,  2.1180427e-03,  8.4224651e-03, ...,
        -4.0007071e-04,  2.3719552e-03, -3.7159508e-03],
       [ 1.2772135e-02, -3.4847087e-03,  2.0979075e-02, ...,
         2.3611756e-02,  2.4236694e-02, -3.7828584e-03],
       [-2.4579108e-02, -7.9067359e-03,  2.5011292e-02, ...,
        -5.2589890e-02,  3.6753878e-02,  3.7756007e-02]], dtype=float32)

In [97]:
# visualizing the name values 
item_names_value

array(['question_id:0', 'question_id:1', 'question_id:2', ...,
       'question_id:72357', 'question_id:72358', 'question_id:72359'],
      dtype='<U17')

## Creating LSH Index Class

In [98]:
class Index_LSH():
  def __init__(self, vectors, labels):
      self.dimension = vectors.shape[1]
      self.vectors = vectors.astype('float32')
      self.labels = labels
  def build_index(self, num_bits=8):
      self.index = faiss.IndexLSH(self.dimension, num_bits)
      self.index.add(self.vectors)
  def results(self, vectors, k=10):
      distances, indices = self.index.search(vectors, k) 
      return [self.labels[i] for i in indices[0]]

## Here I am checking the time taken by LSH to find the similar questions to question with id 5

In [99]:
time_taken = time.time()

lsh_index = Index_LSH(data["vector"], data["name"])
lsh_index.build_index()

vector, name = data['vector'][5:6], data['name'][5]

simlar_question_return_ids = '\n '.join(lsh_index.results(vector))
print(f"Similar questions to question with id: {name} are:\n {simlar_question_return_ids}")

time_2 = time.time()

total_time = time_2-time_taken

print("LSH took {total_time} seconds.".format(total_time=total_time))

Similar questions to question with id: question_id:5 are:
 question_id:139
 question_id:5
 question_id:152
 question_id:132
 question_id:6
 question_id:189
 question_id:203
 question_id:207
 question_id:138
 question_id:23
LSH took 0.022244691848754883 seconds.


# **Exhaustive Search**

Exhaustive Search is a very generic problem solving technique in which all possible element are systematically enumerated for the solution and also checked whether each element satisfies the problem's statement. It requires Linear query time which depends on the size of the dataset and the dimentions. Hence it is very tedious and time taking process.

## Creating Exhaustive Index Class

In [100]:
class Index_ExhaustiveSearch():
  def __init__(self, vectors, labels):
    self.dimension = vectors.shape[1]
    self.vectors = vectors.astype('float32')
    self.labels = labels

  def build_index(self):
    self.index = faiss.IndexFlatL2(self.dimension,)
    self.index.add(self.vectors)

  def results(self, vectors, k=10):
    distances, indices = self.index.search(vectors, k) 
    return [self.labels[i] for i in indices[0]]

## Here I am checking the time taken by Exhaustive Search to find the similar questions to question with id 5

In [101]:
time_taken = time.time()

exi_ind = Index_ExhaustiveSearch(data["vector"], data["name"])
exi_ind.build_index()

vector, name = data['vector'][5:6], data['name'][5]

simlar_question_return_ids = '\n '.join(exi_ind.results(vector))
print(f"Similar questions to question with id: {name} are:\n {simlar_question_return_ids}")

time_2 = time.time()

total_time = time_2-time_taken

print("Exhaustive Search took {total_time} seconds.".format(total_time=total_time))

Similar questions to question with id: question_id:5 are:
 question_id:5
 question_id:262
 question_id:889
 question_id:20491
 question_id:836
 question_id:472
 question_id:151
 question_id:372
 question_id:698
 question_id:6287
Exhaustive Search took 0.035036325454711914 seconds.


# **Trees and Graphs**

## Creating Tree And Graphs Index Class

In [102]:
class Index_Annoy():
    def __init__(self, vectors, labels):
        self.dimention = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels


    def build(self, number_of_trees=5):
        self.index = annoy.AnnoyIndex(self.dimention)
        for i, vec in enumerate(self.vectors):
            self.index.add_item(i, vec.tolist())
        self.index.build(number_of_trees)
        
    def results(self, vector, k=10):
        indices = self.index.get_nns_by_vector(vector.tolist(), k)
        return [self.labels[i] for i in indices]


## Here I am checking the time taken by Tree and Graph Algorithm to find the similar questions to question with id 5

In [103]:
time_taken = time.time()

annoy_ind = Index_Annoy(data["vector"], data["name"])
annoy_ind.build()

vector, name = data['vector'][5], data['name'][5]

simlar_question_return_ids = '\n '.join(annoy_ind.results(vector))
print(f"Similar questions to question with id: {name} are:\n {simlar_question_return_ids}")

time_2 = time.time()

total_time = time_2-time_taken

print("Trees and Graphs algorithm took {total_time} seconds.".format(total_time=total_time))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  if __name__ == '__main__':


Similar questions to question with id: question_id:5 are:
 question_id:5
 question_id:262
 question_id:431
 question_id:2737
 question_id:8984
 question_id:378
 question_id:88
 question_id:585
 question_id:121
 question_id:275
Trees and Graphs algorithm took 0.8027896881103516 seconds.


# **Product Quantization**

Product Quantization is a type of Vector quantization process. It's used to increase the speed of Approximate nearest neighbour search.
Quantization is basically used to considerably reduce or compress data which is very important task when comparing large number of arrays of vectors as they all must be loaded in memory in order to be compared. 
Compared to other quantization methods Product Quantization reduces the memory size effectively.
Key idea behind Product Quantization is:
*   First divide all the vectors into m subvectors called blocks.
*   Then Vector Quantizes each subvector independently.


### Creating Index Class for Product Quantization

In [104]:
class Index_PQ():
    def __init__(self, vectors, labels):
        self.dimension = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels    
        
    def build(self, 
              number_of_partition=8, 
              search_in_x_partitions=2, 
              subvector_size=8):
        quantizer = faiss.IndexFlatL2(self.dimension)
        self.index = faiss.IndexIVFPQ(quantizer, 
                                      self.dimension, 
                                      number_of_partition, 
                                      search_in_x_partitions, 
                                      subvector_size)
        self.index.train(self.vectors)
        self.index.add(self.vectors)
        
    def results(self, vectors, k=10):
        distances, indices = self.index.search(vectors, k) 
        return [self.labels[i] for i in indices[0]]

## Here I am checking the time taken by Product Quantization to find the similar questions to question with id 5

In [106]:
time_taken = time.time()

annoy_ind = Index_PQ(data["vector"], data["name"])
annoy_ind.build()

vector, name = data['vector'][5:6], data['name'][5]

simlar_question_return_ids = '\n '.join(annoy_ind.results(vector))
print(f"Similar questions to question with id: {name} are:\n {simlar_question_return_ids}")

time_2 = time.time()

total_time = time_2-time_taken

print("Product Quantization took {total_time} seconds.".format(total_time=total_time))

Similar questions to question with id: question_id:5 are:
 question_id:5
 question_id:50
 question_id:67
 question_id:2
 question_id:96
 question_id:102
 question_id:114
 question_id:131
 question_id:90
 question_id:46
Product Quantization took 3.5436084270477295 seconds.


# **HNSW**

HNSW stands for Hierarchical Navigable Small World Graphs: These graphs are more recently developed in serach.HNSW based ANNS consistently tops as the highest performing indexes[1]. It has great search-quality, good search-speed, but substantial index sizes. 

## Creating Index Class for HNSW

In [107]:
class Index_HNSW():
  def __init__(self, vectors, labels):
    self.dimention = vectors.shape[1]
    self.vectors = vectors.astype('float32')
    self.labels = labels

  def build(self):
    self.index = nmslib.init(method='hnsw', space='cosinesimil')
    self.index.addDataPointBatch(self.vectors)
    self.index.createIndex({'post': 2})
        
  def results(self, vector, k=10):
    indices = self.index.knnQuery(vector, k=k)
    return [self.labels[i] for i in indices[0]]

## Here I am checking the time taken by HNSW Algorithm to find the similar questions to question with id 5

In [108]:
time_taken = time.time()

hnsw_ind = Index_HNSW(data["vector"], data["name"])
hnsw_ind.build()

vector, name = data['vector'][5:6], data['name'][5]

simlar_question_return_ids = '\n '.join(hnsw_ind.results(vector))
print(f"Similar questions to question with id: {name} are:\n {simlar_question_return_ids}")

time_2 = time.time()

total_time = time_2-time_taken

print("HNSW Algorithm took {total_time} seconds.".format(total_time=total_time))

Similar questions to question with id: question_id:5 are:
 question_id:5
 question_id:262
 question_id:47462
 question_id:889
 question_id:836
 question_id:2
 question_id:20491
 question_id:472
 question_id:372
 question_id:151
HNSW Algorithm took 54.92410588264465 seconds.
